# 爬博客园数据
从博客园网站爬取特定用户的博客数据，并将爬取到的信息保存到本地文件中。

3 种方法只需要掌握 2 种，其中正则是必需的。

In [3]:
import requests
import re
from lxml import etree         # 用于解析 HTML 文档
from bs4 import BeautifulSoup  # CSS 选择器
"""
1、明确需要爬的内容：标题，时间，阅读量，评论数，推荐数，摘要
2、用Edge浏览器打开需要爬虫的网址，右键-检查-网络-清除网络日志-重新加载页面-点“名称”下面的内容-预览(判断我们需要爬取的内容在不在里面)
   -标头(确定请求URL，请求方法（GET，POST），请求参数，请求标头，Cookies，User-Agent信息)-翻页
"""

# 检查网站是否有反爬功能（博客园没有）
url = "https://www.cnblogs.com/pinard?page=1"
res = requests.get(url)
print(f"状态码：{res.status_code}")
print(f"网页编码：{res.encoding}\n")
print(f"网页源代码：\n{res.text}")

状态码：200
网页编码：utf-8

网页源代码：
<!DOCTYPE html>
<html lang="zh-cn">
<head>
    <meta charset="utf-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <meta name="referrer" content="origin-when-cross-origin" />
    
    
    
    <meta http-equiv="Cache-Control" content="no-transform" />
    <meta http-equiv="Cache-Control" content="no-siteapp" />
    <meta http-equiv="X-UA-Compatible" content="IE=edge" />
    <title>刘建平Pinard - 博客园</title>
    <link rel="icon" id="favicon" href="https://assets.cnblogs.com/favicon_v3_2.ico" type="image/x-icon" />
    <link rel="canonical" href="https://www.cnblogs.com/pinard" />
    
    <link rel="stylesheet" href="/css/blog-common.min.css?v=DTx8XrQTETRXxrlh9YxMFto-XNDgQSH6APv3R0bgafA" />
    

    <link id="MainCss" rel="stylesheet" href="/skins/blacklowkey/bundle-blacklowkey.min.css?v=QmhJuqutwsyRVPwV8wOHP8zM5sTwHe5vHM1HxnzZE-8" />
        
    <link type="text/css" rel="stylesheet" href="https://www.cnblogs.com/pinard/c

In [ ]:
# 构建 URL 列表
urls = [f'https://www.cnblogs.com/pinard/default.html?page={page}' for page in range(1, 15)]
print(urls)

## 方法一：正则

In [ ]:
<!-- 源代码 -->
<div class="day" role="article" aria-describedby="postlist_description_11114748">
    <div class="dayTitle">
        <a href="https://www.cnblogs.com/pinard/p/archive/2019/07/01">2019年7月1日
</a>
    </div>

        <div class="postTitle" role="heading" aria-level="2">
            <a class="postTitle2 vertical-middle" href="https://www.cnblogs.com/pinard/p/11114748.html">
    <span>
        XGBoost类库使用小结
    </span>
</a>
        </div>
        <div class="postCon">

<div class="c_b_p_desc" id="postlist_description_11114748">
摘要：        
在XGBoost算法原理小结中，我们讨论了XGBoost的算法原理，这一片我们讨论如何使用XGBoost的Python类库，以及一些重要参数的意义和调参思路。 本文主要参考了XGBoost的Python文档&#160;和&#160;XGBoost的参数文档。 1. XGBoost类库概述 XGBoost除了支持Pyth    <a href="https://www.cnblogs.com/pinard/p/11114748.html" class="c_b_p_desc_readmore">阅读全文</a>
</div>
</div>
        <div class="clear"></div>
        <div class="postDesc">posted @ 2019-07-01 18:10
刘建平Pinard
<span data-post-id="11114748" class="post-view-count">阅读(66627)</span>
<span data-post-id="11114748" class="post-comment-count">评论(135)</span>
<span data-post-id="11114748" class="post-digg-count">推荐(21)</span>
</div>
        <div class="clear"></div>
</div>

提取 1 个 URL 中的内容：

In [8]:
# 初始化文章数量
count = 0

# 发送 GET 请求，获取网页内容
res = requests.get(url)
res.encoding = 'utf-8'
# (.*?)无法匹配换行符，所以要先去除换行符
str_text = res.text.replace("\n", "")

# 保存原始 HTML 内容
# 因有反爬功能存在，返回的内容和我们看到的内容可能不一致，需要保存下来看一下
with open('./output/cnblogs.html', "w", encoding='utf-8') as f:
    f.write(str_text)

# 提取博客条目
# 开始部分：<div class="day"
# 提取部分：(.*?)
# 结尾部分：<div class="clear"></div></div>
blogs = re.findall(r'<div class="day"(.*?)<div class="clear"></div></div>', str_text)
# print(len(blogs), blogs)

# 解析每个博客的详细信息
for blog in blogs:
    # 标题，时间，阅读量，评论数，推荐数，摘要
    title = re.findall(r'<span>(.*?)</span>', blog)[0].strip()
    date = re.findall(r'<div class="dayTitle">.*?>(.*?)</a>', blog)[0]  # .*?>：匹配 <a href="...">，包括前面的空格
    read = re.findall(r'阅读\((.*?)\)', blog)[0]
    comment = re.findall(r'评论\((.*?)\)', blog)[0]
    recommendation = re.findall(r'推荐\((.*?)\)', blog)[0]
    abstract = re.findall(r'摘要：(.*?)<a', blog)[0]
    print(title, date, read, comment, recommendation, abstract, "\n")

    # 连接成一个字符串
    strs = f"{title}\t{date}\t{read}\t{comment}\t{recommendation}\t{abstract}"
    # print(strs)
    
    # 以追加模式将字符串写入文件，每行一个博客记录
    with open('./output/cnblogs_datas.txt', 'a', encoding='utf-8') as f:
        f.write(strs + '\n')
    count += 1
        
print(f"文章数量：{count}")

XGBoost类库使用小结 2019年7月1日 66952 135 21         在XGBoost算法原理小结中，我们讨论了XGBoost的算法原理，这一片我们讨论如何使用XGBoost的Python类库，以及一些重要参数的意义和调参思路。 本文主要参考了XGBoost的Python文档&#160;和&#160;XGBoost的参数文档。 1. XGBoost类库概述 XGBoost除了支持Pyth     

XGBoost算法原理小结 2019年6月5日 78728 201 36         在两年半之前作过梯度提升树(GBDT)原理小结，但是对GBDT的算法库XGBoost没有单独拿出来分析。虽然XGBoost是GBDT的一种高效实现，但是里面也加入了很多独有的思路和方法，值得单独讲一讲。因此讨论的时候，我会重点分析和GBDT不同的地方。 本文主要参考了XGBoost的论文和陈天奇的P     

机器学习中的矩阵向量求导(五) 矩阵对矩阵的求导 2019年5月27日 43324 27 7         在矩阵向量求导前4篇文章中，我们主要讨论了标量对向量矩阵的求导，以及向量对向量的求导。本文我们就讨论下之前没有涉及到的矩阵对矩阵的求导，还有矩阵对向量，向量对矩阵求导这几种形式的求导方法。 本文所有求导布局以分母布局为准，为了适配矩阵对矩阵的求导，本文向量对向量的求导也以分母布局为准，这和前面的文章     

机器学习中的矩阵向量求导(四) 矩阵向量求导链式法则 2019年5月7日 57439 71 29         在机器学习中的矩阵向量求导(三) 矩阵向量求导之微分法中，我们讨论了使用微分法来求解矩阵向量求导的方法。但是很多时候，求导的自变量和因变量直接有复杂的多层链式求导的关系，此时微分法使用起来也有些麻烦。需要一些简洁的方法。 本文我们讨论矩阵向量求导链式法则，使用该法则很多时候可以帮我们快速求出导数结果     

机器学习中的矩阵向量求导(三) 矩阵向量求导之微分法 2019年4月29日 41657 84 19         在机器学习中的矩阵向量求导(二) 矩阵向量求导之定义法中，我们讨论了定义法求解矩阵向量求导的方法，但是这个方法对于比较复杂的求导式子，中间运算会很复杂，同时排列求导出的结果也很麻烦。因此我们需要其他的一些

提取多个 URL 中的内容：

In [7]:
# 初始化文章数量
count = 0

for url in urls:
    # 发送 GET 请求，获取网页内容
    res = requests.get(url)
    res.encoding = 'utf-8'
    # (.*?)无法匹配换行符，所以要先去除换行符
    str_text = res.text.replace("\n", "")

    # 保存原始 HTML 内容
    # 因有反爬功能存在，返回的内容和我们看到的内容可能不一致，需要保存下来看一下
    with open('./output/cnblogs.html', "w", encoding='utf-8') as f:
        f.write(str_text)

    # 提取博客条目
    # 开始部分：<div class="day"
    # 提取部分：(.*?)
    # 结尾部分：<div class="clear"></div></div>
    blogs = re.findall(r'<div class="day"(.*?)<div class="clear"></div></div>', str_text)
    # print(len(blogs), blogs)
    # break
    
    # 解析每个博客的详细信息
    for blog in blogs:
        # 标题，时间，阅读量，评论数，推荐数，摘要
        title = re.findall(r'<span>(.*?)</span>', blog)[0].strip()
        date = re.findall(r'<div class="dayTitle">.*?>(.*?)</a>', blog)[0]  # .*?>：匹配 <a href="...">，包括前面的空格
        read = re.findall(r'阅读\((.*?)\)', blog)[0]
        comment = re.findall(r'评论\((.*?)\)', blog)[0]
        recommendation = re.findall(r'推荐\((.*?)\)', blog)[0]
        abstract = re.findall(r'摘要：(.*?)<a', blog)[0]
        print(title, date, read, comment, recommendation, abstract, "\n")

        # 连接成一个字符串
        strs = f"{title}\t{date}\t{read}\t{comment}\t{recommendation}\t{abstract}"
        # print(strs)
        
        # 以追加模式将字符串写入文件，每行一个博客记录
        with open('./output/cnblogs_datas.txt', 'a', encoding='utf-8') as f:
            f.write(strs + '\n')
        count += 1
        
print(f"文章数量：{count}")

XGBoost类库使用小结 2019年7月1日 66627 135 21         在XGBoost算法原理小结中，我们讨论了XGBoost的算法原理，这一片我们讨论如何使用XGBoost的Python类库，以及一些重要参数的意义和调参思路。 本文主要参考了XGBoost的Python文档&#160;和&#160;XGBoost的参数文档。 1. XGBoost类库概述 XGBoost除了支持Pyth     

XGBoost算法原理小结 2019年6月5日 78411 201 36         在两年半之前作过梯度提升树(GBDT)原理小结，但是对GBDT的算法库XGBoost没有单独拿出来分析。虽然XGBoost是GBDT的一种高效实现，但是里面也加入了很多独有的思路和方法，值得单独讲一讲。因此讨论的时候，我会重点分析和GBDT不同的地方。 本文主要参考了XGBoost的论文和陈天奇的P     

机器学习中的矩阵向量求导(五) 矩阵对矩阵的求导 2019年5月27日 43132 27 7         在矩阵向量求导前4篇文章中，我们主要讨论了标量对向量矩阵的求导，以及向量对向量的求导。本文我们就讨论下之前没有涉及到的矩阵对矩阵的求导，还有矩阵对向量，向量对矩阵求导这几种形式的求导方法。 本文所有求导布局以分母布局为准，为了适配矩阵对矩阵的求导，本文向量对向量的求导也以分母布局为准，这和前面的文章     

机器学习中的矩阵向量求导(四) 矩阵向量求导链式法则 2019年5月7日 57167 71 29         在机器学习中的矩阵向量求导(三) 矩阵向量求导之微分法中，我们讨论了使用微分法来求解矩阵向量求导的方法。但是很多时候，求导的自变量和因变量直接有复杂的多层链式求导的关系，此时微分法使用起来也有些麻烦。需要一些简洁的方法。 本文我们讨论矩阵向量求导链式法则，使用该法则很多时候可以帮我们快速求出导数结果     

机器学习中的矩阵向量求导(三) 矩阵向量求导之微分法 2019年4月29日 41449 84 19         在机器学习中的矩阵向量求导(二) 矩阵向量求导之定义法中，我们讨论了定义法求解矩阵向量求导的方法，但是这个方法对于比较复杂的求导式子，中间运算会很复杂，同时排列求导出的结果也很麻烦。因此我们需要其他的一些

## 方法二：Xpath

1、复制 Xpath 路径：

![](./images/cnblogs01.png)

2、按 Ctrl + F，打开搜索窗口，粘贴 Xpath 路径：

<img src="./images/cnblogs02.png" width=800>

3、在搜索输入框，从右向左删 Xpath 路径，直到出现文章标题；<br>
点击 ↑↓ 按钮，可以选择文章标题。

NOTE: 第 1 个和最后 2 个 div 没有标题，中间 10 个（class="day"）有标题，需要在程序中做判断。

<img src="./images/cnblogs03.png" width=800>

In [9]:
for url in urls:
    # 发送 GET 请求，获取 HTML 网页内容
    res = requests.get(url)
    # 将 HTML 文本解析成可进行 XPath 查询的树结构对象
    xpath_html = etree.HTML(res.text)
    # print(xpath_html)  # <Element html at 0x1fbd379f680>
    
    # 提取博客文章块（返回列表）
    # *：匹配任何元素节点
    # @：选取属性
    # //*[@id="mainContent"]：查找 id 为 mainContent 的任何元素
    blogs = xpath_html.xpath('//*[@id="mainContent"]/div/div')
                            # //*[@id="mainContent"]/div/div[2]/div[2]/a/span  # 第 1 篇博文的 xpath
    # print(len(blogs), blogs)
    """
    13 [<Element div at 0x1fbd3760e80>, <Element div at 0x1fbd3760180>, <Element div at 0x1fbd3763f40>, 
    <Element div at 0x1fbd2e0fd40>, <Element div at 0x1fbd37b7740>, <Element div at 0x1fbd37b7e00>, 
    <Element div at 0x1fbd37b6500>, <Element div at 0x1fbd37b5740>, <Element div at 0x1fbd37b5080>, 
    <Element div at 0x1fbd37b4800>, <Element div at 0x1fbd37b4140>, <Element div at 0x1fbd37b5800>, 
    <Element div at 0x1fbd37b5f80>]
    """
    # break

    for blog in blogs:
        # 标题，时间，阅读量，评论数，推荐数
        # 通过查看 HTML 源代码定位
        title = blog.xpath('./div[2]/a/span/text()')  # 第 2 个 <div> 下的 <a> 的 <span> 的文本内容
        date =  blog.xpath('./div[1]/a/text()')      
        read = blog.xpath('./div[5]/span[1]/text()')
        comment = blog.xpath('./div[5]/span[2]/text()')
        recommendation = blog.xpath('./div[5]/span[3]/text()')
        # 如果 title 非空（成功提取到标题）
        if title:
            # 去掉标题两边的换行符和空格
            title = title[0].strip()
            date = date[0].strip()
            read = read[0]
            comment = comment[0]
            recommendation = recommendation[0]
            print(title, date, read, comment, recommendation) 

XGBoost类库使用小结 2019年7月1日 阅读(66627) 评论(135) 推荐(21)
XGBoost算法原理小结 2019年6月5日 阅读(78411) 评论(201) 推荐(36)
机器学习中的矩阵向量求导(五) 矩阵对矩阵的求导 2019年5月27日 阅读(43132) 评论(27) 推荐(7)
机器学习中的矩阵向量求导(四) 矩阵向量求导链式法则 2019年5月7日 阅读(57167) 评论(71) 推荐(29)
机器学习中的矩阵向量求导(三) 矩阵向量求导之微分法 2019年4月29日 阅读(41449) 评论(84) 推荐(19)
机器学习中的矩阵向量求导(二) 矩阵向量求导之定义法 2019年4月26日 阅读(43034) 评论(46) 推荐(22)
机器学习中的矩阵向量求导(一) 求导定义与求导布局 2019年4月22日 阅读(67509) 评论(19) 推荐(55)
强化学习(十九) AlphaGo Zero强化学习原理 2019年3月27日 阅读(39821) 评论(69) 推荐(14)
强化学习(十八) 基于模拟的搜索与蒙特卡罗树搜索(MCTS) 2019年3月4日 阅读(49710) 评论(29) 推荐(5)
强化学习(十七) 基于模型的强化学习与Dyna算法框架 2019年2月15日 阅读(25460) 评论(26) 推荐(2)
强化学习(十六) 深度确定性策略梯度(DDPG) 2019年2月1日 阅读(123501) 评论(318) 推荐(24)
强化学习(十五) A3C 2019年1月29日 阅读(71528) 评论(144) 推荐(4)
强化学习(十四) Actor-Critic 2019年1月15日 阅读(114927) 评论(148) 推荐(9)
强化学习(十三) 策略梯度(Policy Gradient) 2018年12月18日 阅读(122592) 评论(177) 推荐(14)
强化学习(十二) Dueling DQN 2018年11月8日 阅读(58435) 评论(74) 推荐(5)
强化学习(十一) Prioritized Replay DQN 2018年10月16日 阅读(52696) 评论(153) 推荐(14)
强化学习（十）Double DQN (DDQN) 2018年10月12日 阅读(108696) 

## 方法三：bs4

In [11]:
for url in urls:
    # class可以有多个，id 是唯一的
    # '.'：class 属性
    # 空格代表子节点
    # '#'：id 属性
    # 使用 BeautifulSoup 库来解析 HTML 内容，用的是 lxml 解析器
    bs = BeautifulSoup(res.text, 'lxml')
    blogs = bs.select('.day')
    for blog in blogs:
        title = blog.select('.postTitle span')[0].string.strip()
        date =  blog.select('.dayTitle a')[0].string.strip()
        read = blog.select('.post-view-count')[0].string
        comment = blog.select('.post-comment-count')[0].string
        recommendation = blog.select('.post-digg-count')[0].string
        if title:
            print(title, date, read, comment, recommendation) 

线性回归原理小结 2016年10月28日 阅读(60593) 评论(111) 推荐(24)
精确率与召回率，RoC曲线与PR曲线 2016年10月24日 阅读(42004) 评论(35) 推荐(16)
最小二乘法小结 2016年10月19日 阅读(108285) 评论(72) 推荐(58)
梯度下降（Gradient Descent）小结 2016年10月17日 阅读(542732) 评论(247) 推荐(165)
线性回归原理小结 2016年10月28日 阅读(60593) 评论(111) 推荐(24)
精确率与召回率，RoC曲线与PR曲线 2016年10月24日 阅读(42004) 评论(35) 推荐(16)
最小二乘法小结 2016年10月19日 阅读(108285) 评论(72) 推荐(58)
梯度下降（Gradient Descent）小结 2016年10月17日 阅读(542732) 评论(247) 推荐(165)
线性回归原理小结 2016年10月28日 阅读(60593) 评论(111) 推荐(24)
精确率与召回率，RoC曲线与PR曲线 2016年10月24日 阅读(42004) 评论(35) 推荐(16)
最小二乘法小结 2016年10月19日 阅读(108285) 评论(72) 推荐(58)
梯度下降（Gradient Descent）小结 2016年10月17日 阅读(542732) 评论(247) 推荐(165)
线性回归原理小结 2016年10月28日 阅读(60593) 评论(111) 推荐(24)
精确率与召回率，RoC曲线与PR曲线 2016年10月24日 阅读(42004) 评论(35) 推荐(16)
最小二乘法小结 2016年10月19日 阅读(108285) 评论(72) 推荐(58)
梯度下降（Gradient Descent）小结 2016年10月17日 阅读(542732) 评论(247) 推荐(165)
线性回归原理小结 2016年10月28日 阅读(60593) 评论(111) 推荐(24)
精确率与召回率，RoC曲线与PR曲线 2016年10月24日 阅读(42004) 评论(35) 推荐(16)
最小二乘法小结 2016年10月19日 阅读(108285) 评论(72) 推荐(58)
梯度下降（Gradient 

In [12]:
# ?在正则表达式中有两层含义
# 如果 ? 出现在代码次数的规则后面则代表非贪婪模式，否则代表 0 或 1 次（贪婪模式）
strs = 'accbbcsdabcbc'
print(re.findall(r'a(.*?)c', strs))
print(re.findall(r'a(.*)c', strs))
print(re.findall(r'a(.?)c', strs))   # ['c', 'b']
print(re.findall(r'a(.??)c', strs))  # ['', 'b']  # 在这里第一个问号是代表 0 或 1 次，第二个问号代表非贪婪模式

['', 'b']
['ccbbcsdabcb']
['c', 'b']
['', 'b']
